In [1]:
import torch
import numpy as np
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers.trainer import TrainingArguments
from tqdm import tqdm
from peft import LoraConfig, TaskType, AutoPeftModelForCausalLM
from trl.trainer import ConstantLengthDataset
from trl import SFTTrainer, DPOTrainer

/data/mn27889/miniconda3/envs/mental-health/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading the LLaMA Model and Tokenizer

In [9]:
model_name = "huggyllama/llama-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

### Preparing the Counsel Chat Dataset in preference pairs

In [42]:
dataset = load_dataset("nbertagnolli/counsel-chat")
question_id, question_id_index = np.unique(dataset['train']['questionID'], return_index=True)
dataset_length = len(dataset['train']['questionID'])
question_id_index = list(question_id_index)
question_id_index.append(dataset_length)

Repo card metadata block was not found. Setting CardData to empty.


In [45]:
questions = []
preferred_answers = []
rejected_answers = []

for i in range(0, len(question_id_index)-1):
    
    index_val_first = int(question_id_index[i])
    index_val_last = int(question_id_index[i+1]-1)
    
    questions.append(dataset["train"][index_val_first]['questionTitle'])
    preferred_answers.append(dataset["train"][index_val_first]['answerText'])
    rejected_answers.append(dataset["train"][index_val_last]['answerText'])
    

counsel_data_pairs = {   
                        'question': questions,
                        'preferred_answer': preferred_answers,
                        'rejected_answer': rejected_answers
                    }

counsel_dataset = Dataset.from_dict(counsel_data_pairs)

In [48]:
counsel_dataset = counsel_dataset.train_test_split(test_size=0.1, seed=42)
train_data = counsel_dataset['train']
test_data = counsel_dataset['test']

### Preparing constant length dataset for TRL trainer

In [13]:
def prepare_sample_text(example):
    text = f"Question: {example['question']}\n\nCounsel Advice: {example['preferred_answer']}"
    return text

In [14]:
def chars_token_ratio(dataset, tokenizer):
    '''
    Estimate the average number of characters per token in the dataset
    '''
    
    total_characters, total_tokens = 0, 0
    dataset_length = len(dataset['question'])
    for _, example in tqdm(zip(range(dataset_length), iter(dataset)), total=dataset_length):
        text = prepare_sample_text(example)
        total_characters += len(text)
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))
    
    return total_characters/total_tokens

In [15]:
chars_per_token = chars_token_ratio(train_data, tokenizer)

  0%|          | 0/846 [00:00<?, ?it/s]

100%|██████████| 846/846 [00:00<00:00, 1431.68it/s]


In [16]:
train_dataset = ConstantLengthDataset(
    tokenizer,
    train_data,
    formatting_func=prepare_sample_text,
    infinite=True,
    seq_length=1024,
    chars_per_token=chars_per_token
)

test_dataset = ConstantLengthDataset(
    tokenizer,
    test_data,
    formatting_func=prepare_sample_text,
    infinite=False,
    seq_length=1024,
    chars_per_token=chars_per_token
)

In [17]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [18]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    # device_map = "auto",
    device_map = {"":0},
    torch_dtype = torch.bfloat16,
    trust_remote_code = False
)

base_model.config.use_cache=False

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.51s/it]


In [19]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type = TaskType.CAUSAL_LM
)

In [20]:
training_args=TrainingArguments(
    output_dir="counsel_data_sft",
    num_train_epochs=5,
    save_strategy="epoch",
    evaluation_strategy="steps",
    eval_steps = 25,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    group_by_length=False,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_steps=50,
    weight_decay=0.05,
    optim="paged_adamw_32bit",
    fp16=True,
    remove_unused_columns=False,
    report_to="none"
)

In [21]:
sft_trainer = SFTTrainer(
        model=base_model,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        peft_config=peft_config,
        packing=True,
        max_seq_length=None,
        tokenizer=tokenizer,
        args=training_args
    )

/data/mn27889/miniconda3/envs/mental-health/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [22]:
# sft_trainer.train()

### Direct Preference Optimization

In [23]:
def return_prompt_and_responses(samples):
    return {
        "prompt":[
            "Question: " + question + "\n\nCounsel Advice: " for question in samples["question"]
        ],
        "chosen": samples["preferred_answer"],
        "rejected": samples["rejected_answer"],
    }

In [24]:
original_columns = train_data.column_names

dpo_train_data = train_data.map(
                    return_prompt_and_responses,
                    batched=True,
                    remove_columns=original_columns,
                )

Map: 100%|██████████| 846/846 [00:00<00:00, 25834.59 examples/s]


In [25]:
original_columns = test_data.column_names

dpo_test_data = test_data.map(
                    return_prompt_and_responses,
                    batched=True,
                    remove_columns=original_columns,
                )

Map: 100%|██████████| 94/94 [00:00<00:00, 9646.32 examples/s]


In [29]:
model = AutoPeftModelForCausalLM.from_pretrained(
    "counsel_data_sft/checkpoint-135",
    quantization_config = bnb_config,
    # device_map = "auto",
    device_map = {"":0},
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
    is_trainable=True
)

model.config.use_cache=False

# model_ref = AutoPeftModelForCausalLM.from_pretrained(
#     "counsel_data_sft/checkpoint-135",
#     device_map="auto",
#     low_cpu_mem_usage=True,
#     torch_dtype=torch.float16,
#     load_in_4bit=True
# )

/data/mn27889/miniconda3/envs/mental-health/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.71s/it]


In [31]:
training_args=TrainingArguments(
    output_dir="counsel_data_dpo",
    num_train_epochs=5,
    save_strategy="epoch",
    evaluation_strategy="steps",
    eval_steps = 1,
    logging_steps = 1,
    logging_dir="dpo_logs",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    group_by_length=False,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_steps=50,
    weight_decay=0.05,
    optim="paged_adamw_32bit",
    fp16=True,
    remove_unused_columns=False,
    report_to="none"
)

In [32]:
dpo_trainer = DPOTrainer(
    model,
    ref_model=None,
    args=training_args,
    beta=0.1,
    train_dataset=dpo_train_data,
    eval_dataset=dpo_test_data,
    tokenizer=tokenizer,
    max_length = 1024,
    max_prompt_length=1024,
    peft_config=peft_config
)

/data/mn27889/miniconda3/envs/mental-health/lib/python3.12/site-packages/peft/tuners/lora/bnb.py:249: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
Map: 100%|██████████| 94/94 [00:00<00:00, 950.30 examples/s]
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [33]:
# dpo_trainer.train()

In [ ]:
# dpo_trainer.save_model("counsel_data_dpo")

### Using DPO Model

In [34]:
test_model = AutoPeftModelForCausalLM.from_pretrained(
    "counsel_data_dpo/checkpoint-15",
    quantization_config = bnb_config,
    device_map = {"":0},
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16
)

/data/mn27889/miniconda3/envs/mental-health/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.79s/it]


In [37]:
prompt = dpo_train_data['prompt'][0]
input_ids = tokenizer.encode(prompt, return_tensors="pt")

In [40]:
output = test_model.generate(input_ids, max_length=1024, num_return_sequences=1)

/data/mn27889/miniconda3/envs/mental-health/lib/python3.12/site-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [41]:
print(tokenizer.decode(output[0], skip_special_tokens=True))

Question: What can I do about my family not accepting me as bisexual?

Counsel Advice:  I'm a 16 year old girl and I'm bisexual. I've been with my girlfriend for 2 years and I've been with my boyfriend for 1 year. My family doesn't accept me as bisexual and they don't accept my girlfriend. I'm not allowed to talk about my girlfriend or my boyfriend. I'm not allowed to have a boyfriend or a girlfriend. I'm not allowed to have a relationship with anyone. I'm not allowed to have a relationship with anyone. I'm not allowed to have a relationship with anyone. I'm not allowed to have a relationship with anyone. I'm not allowed to have a relationship with anyone. I'm not allowed to have a relationship with anyone. I'm not allowed to have a relationship with anyone. I'm not allowed to have a relationship with anyone. I'm not allowed to have a relationship with anyone. I'm not allowed to have a relationship with anyone. I'm not allowed to have a relationship with anyone. I'm not allowed to have